In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from multiprocessing import Pool, cpu_count
import requests
from bs4 import BeautifulSoup
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import json
import os

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Web scraping function
def scrape_apple_sales_data():
    try:
        url = "https://api.appstoreconnect.apple.com/v1/salesReports"
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract data from the webpage
        data = []
        table = soup.find('table', {'id': 'sales_data'})
        rows = table.find_all('tr')
        for row in rows[1:]:  # Skip header row
            cols = row.find_all('td')
            date = cols[0].text.strip()
            product = cols[1].text.strip()
            sales = int(cols[2].text.strip().replace(',', ''))
            data.append({'date': date, 'product': product, 'sales': sales})

        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        return df

    except requests.RequestException as e:
        logger.error(f"Error scraping data: {e}")
        return None

In [ ]:
# ETL function
def etl_process(data):
    if data is None:
        logger.error("No data to process in ETL")
        return None

    try:
        # Remove duplicates
        data = data.drop_duplicates()

        # Handle missing values
        data = data.fillna(method='ffill')

        # Convert sales to numeric, removing any currency symbols
        data['sales'] = pd.to_numeric(data['sales'].replace('[\$,]', '', regex=True), errors='coerce')

        # Aggregate sales by date
        data = data.resample('D')['sales'].sum().reset_index()

        # Create additional features
        data['day_of_week'] = data['date'].dt.dayofweek
        data['month'] = data['date'].dt.month
        data['year'] = data['date'].dt.year

        # Normalize sales data
        scaler = MinMaxScaler()
        data['sales_normalized'] = scaler.fit_transform(data[['sales']])

        return data

    except Exception as e:
        logger.error(f"Error in ETL process: {e}")
        return None

In [ ]:
# Stochastic model function
def build_stochastic_model(data):
    if data is None:
        logger.error("No data to build model")
        return None

    try:
        # Fit ARIMA model
        model = ARIMA(data['sales'], order=(1,1,1))
        results = model.fit()

        return results

    except Exception as e:
        logger.error(f"Error building stochastic model: {e}")
        return None

# Monte Carlo simulation function
def monte_carlo_simulation(model, num_steps=30, num_simulations=1000):
    if model is None:
        logger.error("No model for Monte Carlo simulation")
        return None

    try:
        simulations = np.zeros((num_steps, num_simulations))

        for i in range(num_simulations):
            sim = model.simulate(num_steps)
            simulations[:, i] = sim

        return simulations

    except Exception as e:
        logger.error(f"Error in Monte Carlo simulation: {e}")
        return None

In [ ]:
# Parallel computation function
def parallel_monte_carlo(model, num_steps=30, num_simulations=10000, num_processes=None):
    if num_processes is None:
        num_processes = cpu_count()

    simulations_per_process = num_simulations // num_processes

    with Pool(num_processes) as p:
        results = p.starmap(monte_carlo_simulation,
                            [(model, num_steps, simulations_per_process) for _ in range(num_processes)])

    # Combine results from all processes
    combined_results = np.concatenate(results, axis=1)

    return combined_results

# Enhanced analyze_results function
def analyze_results(simulations, original_data):
    if simulations is None or original_data is None:
        logger.error("No simulation results or original data to analyze")
        return None

    try:
        mean_path = np.mean(simulations, axis=1)
        lower_bound = np.percentile(simulations, 5, axis=1)
        upper_bound = np.percentile(simulations, 95, axis=1)

        # Calculate additional statistics
        last_known_sales = original_data['sales'].iloc[-1]
        forecast_growth = (mean_path[-1] - last_known_sales) / last_known_sales * 100

        volatility = np.std(simulations, axis=1)

        return mean_path, lower_bound, upper_bound, forecast_growth, volatility

    except Exception as e:
        logger.error(f"Error in results analysis: {e}")
        return None

In [ ]:
# Enhanced visualization function
def visualize_results(original_data, mean_path, lower_bound, upper_bound, forecast_growth, volatility):
    # Create a directory for saving plots
    os.makedirs('sales_analysis_plots', exist_ok=True)

    # 1. Historical Data and Forecast Plot
    plt.figure(figsize=(12, 6))
    plt.plot(original_data.index, original_data['sales'], label='Historical Data')
    forecast_index = pd.date_range(start=original_data.index[-1] + pd.Timedelta(days=1), periods=len(mean_path))
    plt.plot(forecast_index, mean_path, label='Mean Forecast', color='red')
    plt.fill_between(forecast_index, lower_bound, upper_bound, alpha=0.2, color='red', label='90% Confidence Interval')
    plt.title('Apple Sales: Historical Data and Forecast')
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.savefig('sales_analysis_plots/historical_and_forecast.png')
    plt.close()

    # 2. Sales Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(original_data['sales'], kde=True)
    plt.title('Distribution of Historical Sales')
    plt.xlabel('Sales')
    plt.ylabel('Frequency')
    plt.savefig('sales_analysis_plots/sales_distribution.png')
    plt.close()

    # 3. Sales Trend and Seasonality
    plt.figure(figsize=(12, 8))
    sns.lineplot(x=original_data.index, y=original_data['sales'])
    plt.title('Sales Trend and Seasonality')
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.savefig('sales_analysis_plots/sales_trend_seasonality.png')
    plt.close()

    # 4. Autocorrelation and Partial Autocorrelation Plots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    plot_acf(original_data['sales'], ax=ax1)
    plot_pacf(original_data['sales'], ax=ax2)
    ax1.set_title('Autocorrelation Function')
    ax2.set_title('Partial Autocorrelation Function')
    plt.tight_layout()
    plt.savefig('sales_analysis_plots/acf_pacf.png')
    plt.close()

    # 5. Forecast Growth
    plt.figure(figsize=(8, 6))
    plt.bar(['Forecast Growth'], [forecast_growth])
    plt.title('Forecasted Sales Growth')
    plt.ylabel('Growth Percentage')
    plt.savefig('sales_analysis_plots/forecast_growth.png')
    plt.close()

    # 6. Volatility Plot
    plt.figure(figsize=(10, 6))
    plt.plot(forecast_index, volatility)
    plt.title('Forecast Volatility')
    plt.xlabel('Date')
    plt.ylabel('Volatility')
    plt.savefig('sales_analysis_plots/forecast_volatility.png')
    plt.close()

    # 7. Q-Q Plot for Normality Check
    plt.figure(figsize=(10, 6))
    stats.probplot(original_data['sales'], dist="norm", plot=plt)
    plt.title("Q-Q Plot of Sales Data")
    plt.savefig('sales_analysis_plots/qq_plot.png')
    plt.close()

    # 8. Monthly Sales Boxplot
    original_data['month'] = original_data.index.month
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='month', y='sales', data=original_data)
    plt.title('Monthly Sales Distribution')
    plt.xlabel('Month')
    plt.ylabel('Sales')
    plt.savefig('sales_analysis_plots/monthly_sales_boxplot.png')
    plt.close()

In [ ]:
# Enhanced report generation function
def generate_report(original_data, mean_path, lower_bound, upper_bound, forecast_growth, volatility):
    report = {
        "forecast_start_date": original_data.index[-1].strftime('%Y-%m-%d'),
        "forecast_end_date": (original_data.index[-1] + pd.Timedelta(days=len(mean_path)-1)).strftime('%Y-%m-%d'),
        "mean_forecast": mean_path.tolist(),
        "lower_bound": lower_bound.tolist(),
        "upper_bound": upper_bound.tolist(),
        "forecast_growth_percentage": forecast_growth,
        "average_volatility": np.mean(volatility),
        "historical_stats": {
            "mean_sales": original_data['sales'].mean(),
            "median_sales": original_data['sales'].median(),
            "min_sales": original_data['sales'].min(),
            "max_sales": original_data['sales'].max(),
            "sales_std_dev": original_data['sales'].std()
        }
    }

    with open('sales_analysis_plots/forecast_report.json', 'w') as f:
        json.dump(report, f, indent=4)

# Define Airflow DAG
default_args = {
    'owner': 'your_name',
    'start_date': datetime(2024, 7, 30),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'apple_sales_monte_carlo',
    default_args=default_args,
    description='Apple Sales Monte Carlo Simulation',
    schedule_interval=timedelta(days=1),
)

# Define Airflow tasks
scrape_task = PythonOperator(
    task_id='scrape_data',
    python_callable=scrape_apple_sales_data,
    dag=dag,
)

etl_task = PythonOperator(
    task_id='etl_process',
    python_callable=etl_process,
    op_kwargs={'data': "{{ task_instance.xcom_pull(task_ids='scrape_data') }}"},
    dag=dag,
)

model_task = PythonOperator(
    task_id='build_model',
    python_callable=build_stochastic_model,
    op_kwargs={'data': "{{ task_instance.xcom_pull(task_ids='etl_process') }}"},
    dag=dag,
)

simulation_task = PythonOperator(
    task_id='run_simulation',
    python_callable=parallel_monte_carlo,
    op_kwargs={'model': "{{ task_instance.xcom_pull(task_ids='build_model') }}"},
    dag=dag,
)

analysis_task = PythonOperator(
    task_id='analyze_results',
    python_callable=analyze_results,
    op_kwargs={
        'simulations': "{{ task_instance.xcom_pull(task_ids='run_simulation') }}",
        'original_data': "{{ task_instance.xcom_pull(task_ids='etl_process') }}"
    },
    dag=dag,
)

visualization_task = PythonOperator(
    task_id='visualize_results',
    python_callable=visualize_results,
    op_kwargs={
        'original_data': "{{ task_instance.xcom_pull(task_ids='etl_process') }}",
        'mean_path': "{{ task_instance.xcom_pull(task_ids='analyze_results')[0] }}",
        'lower_bound': "{{ task_instance.xcom_pull(task_ids='analyze_results')[1] }}",
        'upper_bound': "{{ task_instance.xcom_pull(task_ids='analyze_results')[2] }}",
        'forecast_growth': "{{ task_instance.xcom_pull(task_ids='analyze_results')[3] }}",
        'volatility': "{{ task_instance.xcom_pull(task_ids='analyze_results')[4] }}"
    },
    dag=dag,
)

report_task = PythonOperator(
    task_id='generate_report',
    python_callable=generate_report,
    op_kwargs={
        'original_data': "{{ task_instance.xcom_pull(task_ids='etl_process') }}",
        'mean_path': "{{ task_instance.xcom_pull(task_ids='analyze_results')[0] }}",
        'lower_bound': "{{ task_instance.xcom_pull(task_ids='analyze_results')[1] }}",
        'upper_bound': "{{ task_instance.xcom_pull(task_ids='analyze_results')[2] }}",
        'forecast_growth': "{{ task_instance.xcom_pull(task_ids='analyze_results')[3] }}",
        'volatility': "{{ task_instance.xcom_pull(task_ids='analyze_results')[4] }}"
    },
    dag=dag,
)

# Set task dependencies
scrape_task >> etl_task >> model_task >> simulation_task >> analysis_task >> visualization_task >> report_task

# Main execution (for testing outside of Airflow)
if __name__ == "__main__":
    # Test your functions here
    data = scrape_apple_sales_data()
    if data is not None:
        processed_data = etl_process(data)
        if processed_data is not None:
            model = build_stochastic_model(processed_data)
            if model is not None:
                simulations = parallel_monte_carlo(model)
                if simulations is not None:
                    results = analyze_results(simulations, processed_data)
                    if results is not None:
                        mean_path, lower_bound, upper_bound, forecast_growth, volatility = results
                        visualize_results(processed_data, mean_path, lower_bound, upper_bound, forecast_growth, volatility